In [3]:
import pylhe
import numpy as np
import math
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
class LHEdataset:
    
    def __init__(self,path,Nevents,xsect,mx):
        self.path = path
        self.Nevents = Nevents
        self.xsect = xsect
        self.mx = mx
        self.lhereader = pylhe.readLHE(path)
        
        # a variable to keep the four momenta of the electron, muon, and X
        self.four_momenta = np.zeros((Nevents,3,4),dtype="float64")

        # a variable to keep the converted data info: 
        # (E,p,theta_phi) for elec, mu, X, and relative angles,
        # theta_{emu}, theta_{eX}, theta_{muX}
        # angle between e+e- decay products of X
        self.conv_data = np.zeros((Nevents,21),dtype="float64")
        
        self.read_lhe()
        self.convert_4_momenta()
                
        
    def read_lhe(self):
        for iev, event in enumerate(self.lhereader):
            for part in event.particles:
                if abs(part.id) == 11 and part.status==1:
                    self.four_momenta[iev,0] = np.array([part.e,part.px,part.py,part.pz])
                if abs(part.id) == 13 and part.status==1:
                    self.four_momenta[iev,1] = np.array([part.e,part.px,part.py,part.pz])
                if abs(part.id) == 103 and part.status==1:
                    self.four_momenta[iev,2] = np.array([part.e,part.px,part.py,part.pz])
    
    # get momentum, and angles, as well as relative angles between particles
    def convert_4_momenta(self):
        # set electron E,p,theta,phi
        self.conv_data[:,0] = self.four_momenta[:,0,0]
        self.conv_data[:,1] = np.linalg.norm(self.four_momenta[:,0,1:4],axis=1)
        self.conv_data[:,2] = np.arccos(self.four_momenta[:,0,3] / self.conv_data[:,1])
        self.conv_data[:,3] = np.arctan2(self.four_momenta[:,0,2], self.four_momenta[:,0,1])
        
        # set muon E,p,theta,phi
        self.conv_data[:,4] = self.four_momenta[:,1,0]
        self.conv_data[:,5] = np.linalg.norm(self.four_momenta[:,1,1:4],axis=1)
        self.conv_data[:,6] = np.arccos(self.four_momenta[:,1,3] / self.conv_data[:,5])
        self.conv_data[:,7] = np.arctan2(self.four_momenta[:,1,2], self.four_momenta[:,1,1])

        # set X E,p,theta,phi        
        self.conv_data[:,8] = self.four_momenta[:,2,0]
        self.conv_data[:,9] = np.linalg.norm(self.four_momenta[:,2,1:4],axis=1)
        self.conv_data[:,10] = np.arccos(self.four_momenta[:,2,3] / self.conv_data[:,9])
        self.conv_data[:,11] = np.arctan2(self.four_momenta[:,2,2], self.four_momenta[:,2,1])
        
        # set relative angles theta_{emu}, theta_{eX}, theta_{muX}        
        self.conv_data[:,12] = np.einsum("ij,ij->i",self.four_momenta[:,0,1:4],self.four_momenta[:,1,1:4])/self.conv_data[:,1]/self.conv_data[:,5]
        self.conv_data[:,13] = np.einsum("ij,ij->i",self.four_momenta[:,1,1:4],self.four_momenta[:,2,1:4])/self.conv_data[:,1]/self.conv_data[:,9]
        self.conv_data[:,14] = np.einsum("ij,ij->i",self.four_momenta[:,1,1:4],self.four_momenta[:,2,1:4])/self.conv_data[:,5]/self.conv_data[:,9]      
        
        # set angle between e+e- decay products of X. Use approximation theta_{ee} = 2/gamma_X
        self.conv_data[:,15] = np.arccos(np.random.uniform(-1.0,1.0,size=(Nevents,))) # rest frame theta
        self.conv_data[:,16] = np.random.uniform(-np.pi*0.5, np.pi*0.5, size=(Nevents,)) # phi
        self.conv_data[:,17] = np.arccos(boost(self.conv_data[:,15],mx,self.conv_data[:,8])) # theta_lab1
        self.conv_data[:,18] = np.arccos(boost(self.conv_data[:,15]+np.pi,mx,self.conv_data[:,8])) #theta_lab2
        self.conv_data[:,19] = self.conv_data[:,17] + self.conv_data[:,18] # open angle
        self.conv_data[:,20] = energy_boost(self.conv_data[:,15],mx, self.conv_data[:,8]) # elab 1
        self.conv_data[:,21] = energy_boost(self.conv_data[:,15]+np.pi,mx,self.conv_data[:,8]) # elab 2
        


In [5]:
luminosity = 1.5e04
event_path = "/Users/isaac/Work/MUonE/MG5_events/20211215/Events/"
m_e = 5.11e-04
meter = 1
GeV = 1 / (1.97 * meter * 1.0e-16)
Radius = 0.05

angle_cut = 1.0 / 1000.0  # input mrad, change to rad
L_min = 0.02
L_max = 14.5 / 100.0  # input cm, change to m

def decay_rate(mass, g_e):
    alpha_e = (g_e ** 2) / (4 * np.pi)
    return (
        alpha_e
        * mass
        * (1 + 2 * ((m_e ** 2) / (mass ** 2)))
        * np.sqrt(1 - 4 * ((m_e ** 2) / (mass ** 2)))
        * GeV
        / 3
    )


def distance(momentum, mass, g_e):
    return momentum / (decay_rate(mass, g_e) * mass)


def boost(theta, mass, energy):
    beta = np.sqrt(energy ** 2.0 - mass ** 2.0) / energy
    gamma = energy / mass
    pcom = np.sqrt(mass ** 2.0 - 4.0 * (m_e ** 2.0)) / 2.0
    nominator = gamma * (pcom * np.cos(theta) + beta * mass / 2.0)
    denominator = np.sqrt(
        (pcom ** 2.0) * (np.sin(theta) ** 2.0)
        + (gamma ** 2.0) * ((pcom * np.cos(theta) + beta * mass * 0.5) ** 2.0)
    )
    return nominator / denominator


def energy_boost(theta, mass, energy):
    beta = np.sqrt(energy ** 2.0 - mass ** 2.0) / energy
    gamma = energy / mass
    pcom = np.sqrt(mass ** 2.0 - 4.0 * (m_e ** 2.0)) / 2.0
    return gamma * (0.5 * mass + beta * pcom * np.cos(theta))


def read_x_section(mass):
    """Read the cross section from madgraph banner file"""
    banner_file_subfolder = "X_mass_" + str(mass)
    banner_file_name = "X_mass_" + str(mass) + "_tag_1_banner.txt"
    banner_file_path = os.path.join(event_path, banner_file_subfolder, banner_file_name)
    with open(banner_file_path) as banner_file:
        for line in banner_file:
            if line.startswith("#  Integrated weight (pb)"):
                for x in line:
                    if x in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                        index = line.index(x)
                        break
                x_section_value = float(line[index:])
    return x_section_value

In [6]:
sigmalist=[]
mass_list = np.logspace(np.log10(0.002),np.log10(0.2),30).tolist()
mass_list = [round(x,4) for x in mass_list]
for mass in mass_list:
    sigmalist.append(read_x_section(mass))

In [ ]:
def BCBF(axis,coupling,mass,)

In [ ]:
lhe_file = "/Users/isaac/Work/MUonE/MG5_events/20211215/LHE/X_mass_0.002.lhe"
LHE = LHEdataset(lhe_file,100000,sigmalist[0],0.002)
coupling = 1.0e-04
event_number = 0
axis = 1.

    

In [ ]:
datasets_path = "/Users/isaac/Work/MUonE/MG5_events/20211215/LHE/"
for index in range(0,30):
    mass = mass_list[index]
    sigma = sigmalist[index]
    lhefile_name = "X_mass_" + str(mass) + ".lhe"
    LHE = LHEdataset(datasets_path+lhefile_name,100000,sigma,mass)
    for coupling in coupling_list